In [1]:
%run preprocessing.py

import data
merge orders and order_products__prior
features extraction...
size of prd features : 9
size of users features : 18
nb of usersXproducts features : 12
merge prod, user and usersXprod feature on usersXprod
merge train reordered to usersXprod
split train and test
size of training set : 8474661
size of testing set : 4833292
Size of features of dataset : 36
Class weight of 0 : 0.9021997458069414
Class weight of 1 : 0.09780025419305857


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils


Using TensorFlow backend.


In [3]:
Y = np_utils.to_categorical(y, 2)

X_train, X_val, y_train, y_val = train_test_split(np.array(X), Y, test_size = 0.2, random_state = 0)
print("Shape of X :", X_train.shape)
print("Shape of Y :", y_train.shape)

Shape of X : (6779728, 36)
Shape of Y : (6779728, 2)


In [4]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

In [5]:
X_train_new, y_train_new = prepare_sequences(X_train, y_train, 36)
X_train_new = X_train_new.reshape(X_train_new.shape[0], X_train_new.shape[1], 1)

print("Shape of sequence_X :", X_train_new.shape)
print("Shape of sequence_y :", y_train_new.shape)

Shape of sequence_X : (6779728, 36, 1)
Shape of sequence_y : (6779728, 2)


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

max_len = X_train_new.shape[1]
batch_size = 1024
nb_epoch = 100

print('Building STATELESS model...')
model = Sequential()
model.add(LSTM(256, input_shape = (max_len, 1), return_sequences = True, stateful = False))
model.add(LSTM(512, return_sequences = True))
model.add(LSTM(512))
model.add(Dense(2, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
model.summary()

Building STATELESS model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 36, 256)           264192    
_________________________________________________________________
lstm_15 (LSTM)               (None, 36, 512)           1574912   
_________________________________________________________________
lstm_16 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 3,939,330
Trainable params: 3,939,330
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train_new, y_train_new,
                    batch_size = batch_size,  
                    epochs = nb_epoch,  
                    verbose = 1,  
                    validation_split = 0.1,  
                    class_weight = {0: 1, 1: 2.7})

loss = history.history.get('loss')
acc = history.history.get('acc')

''' Access the performance on validation data '''
val_loss = history.history.get('val_loss')
val_acc = history.history.get('val_acc')

''' Visualize the loss and accuracy of both models'''
import matplotlib.pyplot as plt
plt.figure(0)
plt.subplot(121)
plt.plot(range(len(loss)), loss,label = 'Training')
plt.plot(range(len(val_loss)), val_loss,label = 'Validation')
plt.title('Loss')
plt.legend(loc='upper left')
plt.subplot(122)
plt.plot(range(len(acc)), acc,label = 'Training')
plt.plot(range(len(val_acc)), val_acc,label = 'Validation')
plt.title('Accuracy')
plt.show()

In [ ]:
X_val_new, y_val_new = prepare_sequences(X_val, y_val, 17)
X_val_new = X_val_new.reshape(X_val_new.shape[0], X_val_new.shape[1], 1)
print("Shape of sequence_X :", X_val_new.shape)
print("Shape of sequence_y :", y_val_new.shape)

### Final evaluation of the model
scores = model.evaluate(X_val_new, y_val_new, verbose = 0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
y_pred = model.predict(X_val_new, batch_size = 32, verbose = 0)

y_pred = pd.DataFrame(y_pred, columns = ["no", "yes"])
y_val = pd.DataFrame(y_val_new, columns = ["no", "yes"])
print('Misclassified samples: %d' % (y_val.no != y_pred.no).sum())
print('Accuracy : %.2f' % ((y_val.no == y_pred.no).sum() / y_val.no.shape[0]))
print('Accuracy (sklearn): %.2f' % accuracy_score(y_val.no, y_pred.no))
print("==confusion matrix==")
print(confusion_matrix(y_val.no, y_pred.no, labels = [0,1], sample_weight = None))